In [ ]:
#  从视频流中采集目标对象的人脸，用于训练分类模型，这个模型用于主程序中摄像头实时视频中人脸识别出目标对象
#  每1帧采集一张人脸，一共采集100个目标对象的样本，使用mtcnn对采集帧进行人脸检测和对齐

import cv2
import sys
import os
import tensorflow as tf
import numpy as np
import align.detect_face
import facenet

video_capture = cv2.VideoCapture(0)
capture_interval = 1
capture_num = 100
capture_count = 0
frame_count = 0
detect_multiple_faces = False #因为是训练目标对象，一次只有一张人脸

#这里引用facenet/src/align/align_dataset_mtcnn.py文件的代码对采集帧进行人脸检测和对齐
minsize = 20 # minimum size of face
threshold = [ 0.6, 0.7, 0.7 ]  # three steps's threshold
factor = 0.709 # scale factor
        
with tf.Graph().as_default():
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
    sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
    with sess.as_default():
        pnet, rnet, onet = align.detect_face.create_mtcnn(sess, None)

while True:

    ret, frame = video_capture.read()
    
    #每1帧采集一张人脸，这里采样不进行灰度变换，直接保存彩色图
    if(capture_count%capture_interval == 0): 
        
        bounding_boxes, _ = align.detect_face.detect_face(frame, minsize, pnet, rnet, onet, threshold, factor)
        nrof_faces = bounding_boxes.shape[0]
                          
        for face_position in bounding_boxes: #因为只采集一张人脸，所以实际只遍历一次
            face_position=face_position.astype(int)
            cropped = frame[face_position[1]:face_position[3],face_position[0]:face_position[2],:]
            scaled = cv2.resize(cropped, (160, 160), interpolation=cv2.INTER_CUBIC )  #这里取和负样本一样大小
            cv2.imwrite('/Users/admin/Desktop/abby'+str(frame_count) + '.jpg', scaled)
                                
        frame_count += 1
          
    capture_count += 1
   
    if frame_count >= capture_num:
        break

video_capture.release()
cv2.destroyAllWindows()
print('采集完成')